
1. 2段目で1minごとの予測モデルを作る
    - 15分周期とかの情報を入れてあげれば分布の偏りも含めてモデルが学習してくれそう？
2. 後処理
    1. （必要があれば予測値のキャリブレーション）
    2. 以下を繰り返す
        1. metric に合わせて N分30秒のように表現できる１分おきの地点での期待値のようなスコアを予測値から求める
            - 1, 3, 5, 7.5, 10, 12.5, 15, 20, 25, 30 分以内の予測に対して 10,9,8,...1 と重みがつく
            - ２回目以降は差分計算で定数時間で求まる
        2. スコアが最も高いものをピックアップしてeventを作る(その地点を提出用eventとする)
        3. 作ったeventにマッチする可能性のある予測確率を割り引く
        
      
さっきは1分ごとで簡略化して説明したけど、ほんとは tolerance 7.5min とかのために30秒ごとで計算したほうがいい (N分30秒じゃなくて、N分15秒かN分45秒で提出したい)
もしくは、1分ごとで最適イベントを作ったあとに、7.5 min と 12.5min の範囲で、15秒ずらしたときに新しくマッチするイベントを見て、よいほうを選ぶか  
        
1回目の提出が tolerance 36 以内でマッチするイベントに関しては、他の提出がそれに tolerance 36 以内でマッチする確率が0になると考える。
つまり、1回目の提出の tolerance 12 以内のやつは、他の提出が 12 以内でマッチする確率が0になる。 (マッチ上書きしても意味ない)
1回目の提出の tolerance 36 以内のやつは、他の提出が 36 以内でマッチする確率が0になる。

In [2]:
import sys, os
sys.path.append(os.pardir)

In [ ]:
from pathlib import Path
import numpy as np
import polars as pl
import os
from hydra import initialize, compose

with initialize(config_path="../run/conf", version_base=None):
    cfg = compose("cv_train")

In [3]:
from src.utils.metrics import event_detection_ap
from src.utils.periodicity import get_periodicity_dict
from src.utils.common import trace
from src.utils.post_process import make_submission

periodicity_dict = get_periodicity_dict(cfg)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
train_df = pl.read_parquet(Path(cfg.dir.data_dir) / "train_series.parquet")
train_df = train_df.with_columns(
            pl.col("timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%z")
        )

In [5]:
event_df = pl.read_csv(Path(cfg.dir.data_dir) / "train_events.csv").drop_nulls()
event_df = event_df.with_columns(
    pl.col("timestamp").str.to_datetime("%Y-%m-%dT%H:%M:%S%z")
)

In [6]:
pred_df = pl.read_parquet("156_gru_transformer_residual.parquet")
pred_all_df = pl.concat([train_df, pred_df], how="horizontal")

In [7]:
pred_all_df.head()

series_id,step,timestamp,anglez,enmo,row_id,prediction_onset,prediction_wakeup
str,u32,"datetime[μs, UTC]",f32,f32,f64,f32,f32
"""038441c925bb""",0,2018-08-14 19:30:00 UTC,2.6367,0.0217,0.0,6.2617e-13,0.000028
"""038441c925bb""",1,2018-08-14 19:30:05 UTC,2.6368,0.0215,1.0,9.6546e-13,0.000034
"""038441c925bb""",2,2018-08-14 19:30:10 UTC,2.637,0.0216,2.0,1.1043e-12,0.000034
"""038441c925bb""",3,2018-08-14 19:30:15 UTC,2.6368,0.0213,3.0,7.2700e-13,0.000034
"""038441c925bb""",4,2018-08-14 19:30:20 UTC,2.6368,0.0215,4.0,7.1623e-13,0.000032


In [1]:
pred_df = (
    pl.read_parquet("./pred_onset.parquet")
    .rename({"label_pred": "stacking_prediction_onset"})
    .drop("label")
    .join(
        pl.read_parquet("./pred_wakeup.parquet")
        .rename({"label_pred": "stacking_prediction_wakeup"})
        .drop("label"),
        on=["series_id", "step"],
        how="left",
    )
)
pred_df = pred_df.with_columns(
    ((pl.col("step") - pl.col("step").shift(1)) != 12)
    .cast(int)
    .cumsum()
    .over("series_id")
    .fill_null(0)
    .alias("chunk_id")
)

NameError: name 'pl' is not defined

In [13]:
def remove_periodicity(
    df,
    periodicity_dict,
    event2col: dict[str, str] = {"onset": "prediction_onset", "wakeup": "prediction_wakeup"},
):
    dfs = []
    for series_id, series_df in df.group_by('series_id'):
        for event, event_pred_col in event2col.items():
            series_df.with_columns(
                pl.Series(name=event_pred_col, values=series_df.get_column(event_pred_col).to_numpy() * (1 - periodicity_dict[series_id] ))
            )
        dfs.append(series_df)
    return pl.concat(dfs)

df = remove_periodicity(series_df, periodicity_dict)
df.head()

series_id,step,timestamp,anglez,enmo,row_id,prediction_onset,prediction_wakeup
str,u32,"datetime[μs, UTC]",f32,f32,f64,f32,f32
"""fe90110788d2""",0,2017-08-04 21:30:00 UTC,-27.707001,0.0298,1.2735396e8,7.6921e-12,0.000122
"""fe90110788d2""",1,2017-08-04 21:30:05 UTC,-33.8675,0.0488,1.27353961e8,8.7983e-12,0.000133
"""fe90110788d2""",2,2017-08-04 21:30:10 UTC,-15.475,0.1077,1.27353962e8,7.8078e-12,0.000125
"""fe90110788d2""",3,2017-08-04 21:30:15 UTC,-73.656197,0.053,1.27353963e8,5.2174e-12,0.000128
"""fe90110788d2""",4,2017-08-04 21:30:20 UTC,-53.152901,0.0601,1.27353964e8,3.7010e-12,0.00011


In [177]:
# 後処理
"""
- 入力：モデルの1分毎の予測値

- onset, wakeup ごとに処理を行う

"""
from tqdm.auto import tqdm


def post_process(
    pred_df,
    event_rate: int | float = 0.005,
    height: float = 0.1,
    event2col: dict[str, str] = {"onset": "prediction_onset", "wakeup": "prediction_wakeup"},
    use_weight = False
):
    """
    1分ごとの予測値を用いてイベントを検出する
    # TODO: 1段目のモデルを入れる場合は1分ごとの予測値をそのまま使わずに周辺の予測値をrollする方が良さそう？

    用語
    - 予測地点: 1段目 or 2段目のモデルによって得られた1分毎の予測位置
    - 候補地点: event の候補となる 15秒 or 45秒始まりの30秒間隔の位置

    Args:
        pred_df (pl.DataFrame): series_id, step, event2colのvalue をカラムに持つ。stepが12の倍数以外の行は無視される。periodicityは削除済み想定
        event_rate (int | float, optional): [0,1) の値であれば1分間に何回イベントが起こるか。intの場合はseries_idごとに同じイベント数を検出。 Defaults to 0.005.
        height (float, optional): 候補地点の期待値がこの値を下回ったら終了。 Defaults to 0.1.
    Returns:
        event_df (pl.DataFrame): row_id, series_id, step, event, score をカラムに持つ。
    """
    high_match_nums = [1, 3, 5, 8, 10, 13, 15, 20, 25, 30]
    low_match_nums = [1, 3, 5, 7, 10, 12, 15, 20, 25, 30]
    match_sums = [2, 6, 10, 15, 20, 25, 30, 40, 50, 60] if use_weight else np.ones(10)
    total_num = sum(high_match_nums + low_match_nums)
    result_events_records = []

    # event ごとに処理
    for event, event_pred_col in event2col.items():
        """
        元の系列の予測地点(長さN): 0, 12, 24, 36, 48, 60, 72, 84, 96, 108, ..., (N-1)*12
        15秒から30秒おきのevent候補地点(長さ2N): 3, 9, 15, 21, 27, 33, 39, 45, 51, 57, ..., (N-1)*12+3, (N-1)*12+9
            - 15秒(3step)から1分おき(長さN): 3, 15, 27, 39, 51, 63, 75, 87, 99, 111, ..., (N-1)*12+3
                - 左の個数 {12: 1, 36: 3, 60: 5, 90: *8*, 120: 10, 150: *13*, 180: 15, 240: 20, 300: 25, 360: 30} high_match_nums
                - 右の個数 {12: 1, 36: 3, 60: 5, 90: *7*, 120: 10, 150: *12*, 180: 15, 240: 20, 300: 25, 360: 30} low_match_nums
            - 45秒(9step)から1分おき(長さN): 9, 21, 33, 45, 57, 69, 81, 93, 105, 117, ..., (N-1)*12+9
                - 左の個数 {12: 1, 36: 3, 60: 5, 90: *7*, 120: 10, 150: *12*, 180: 15, 240: 20, 300: 25, 360: 30} low_match_nums
                - 右の個数 {12: 1, 36: 3, 60: 5, 90: *8*, 120: 10, 150: *13*, 180: 15, 240: 20, 300: 25, 360: 30} high_match_nums       
        """

        # 12の倍数以外の行は無視して1分毎の予測値にする
        minute_pred_df = pred_df.filter(pl.col("step") % 12 == 0)
        max_event_per_series = event_rate if isinstance(event_rate, int) else int(len(minute_pred_df) * event_rate)

        # series_id, step でソート
        minute_pred_df = minute_pred_df.sort(["series_id", "step"])

        # 1. 期待値の計算
        # 1.1 左側を計算 (同じindexの予測を含む左側を計算)
        """
        以下をそれぞれ計算する
        - 15秒(3step)から1分おき(長さN)での候補地点での期待値: stepは 3, 15, 27, 39, 51, 63, 75, 87, 99, 111, ..., (N-1)*12+3
        - 45秒(9step)から1分おき(長さN)での候補地点での期待値: stepは 9, 21, 33, 45, 57, 69, 81, 93, 105, 117, ..., (N-1)*12+9
        計算は左側の予測地点の数と、右側の予測地点の数
        """
        minute_pred_df = minute_pred_df.with_columns(
            pl.sum_horizontal(
                [
                    (
                        pl.col(event_pred_col)
                        .rolling_sum(window_size=window, center=False, min_periods=1)
                        .over("series_id") / match_sums[i]
                    )
                    for i, window in enumerate(high_match_nums)
                ]
            ).alias(f"{event}_left_expectation_plus_3step"),
            pl.sum_horizontal(
                [
                    (
                        pl.col(event_pred_col)
                        .rolling_sum(window_size=window, center=False, min_periods=1)
                        .over("series_id") / match_sums[i]
                    )
                    for i, window in enumerate(low_match_nums)
                ]
            ).alias(f"{event}_left_expectation_plus_9step"),
        )

        # 1.2 右側を計算(同じindexの予測を含まない右側を計算。逆順にして一個ずらしrolling_sumを取る必要がある）
        minute_pred_df = minute_pred_df.reverse()
        minute_pred_df = minute_pred_df.with_columns(
            pl.sum_horizontal(
                [
                    (
                        pl.col(event_pred_col)
                        .shift(1)
                        .rolling_sum(window_size=window, center=False, min_periods=1)
                        .over("series_id")
                        .fill_null(0) / match_sums[i]
                    )
                    for i, window in enumerate(low_match_nums)
                ]
            ).alias(f"{event}_right_expectation_plus_3step"),
            pl.sum_horizontal(
                [
                    (
                        pl.col(event_pred_col)
                        .shift(1)
                        .rolling_sum(window_size=window, center=False, min_periods=1)
                        .over("series_id")
                        .fill_null(0) / match_sums[i]
                    )
                    for i, window in enumerate(high_match_nums)
                ]
            ).alias(f"{event}_right_expectation_plus_9step"),
        )
        minute_pred_df = minute_pred_df.reverse()

        # 合計の期待値計算
        minute_pred_df = minute_pred_df.with_columns(
            (pl.col(f"{event}_left_expectation_plus_3step") + pl.col(f"{event}_right_expectation_plus_3step")).alias(
                f"{event}_expectation_sum_3step"
            ),
            (pl.col(f"{event}_left_expectation_plus_9step") + pl.col(f"{event}_right_expectation_plus_9step")).alias(
                f"{event}_expectation_sum_9step"
            ),
        )

        # 3. 最大値の取得 & 期待値の割引
        """
        各予測地点の power を管理する。powerは以下の11種類
        0: その予測地点が影響を与える範囲は無い
        1: その予測地点が影響を与える範囲は左右1つ(1min)
        2: その予測地点が影響を与える範囲は左右3つ
        ︙
        10: 左右30(step 0~360)

        event を作るたびに、eventからtolerance内にある予測地点のpowerを下げる。
        その際に予測地点からtolerance内にある、eventがあったところも含めた候補地点の期待値を割り引く。
        """
        for series_id, series_df in tqdm(
            minute_pred_df.select(
                ["series_id", event_pred_col, f"{event}_expectation_sum_3step", f"{event}_expectation_sum_9step"]
            ).group_by("series_id"),
            desc="find peaks",
            leave=False,
            total=len(minute_pred_df["series_id"].unique()),
        ):
            preds = series_df[event_pred_col].to_numpy()
            expectation_sum_3step = series_df[f"{event}_expectation_sum_3step"].to_numpy(writable=True)
            expectation_sum_9step = series_df[f"{event}_expectation_sum_9step"].to_numpy(writable=True)
            powers = np.ones(len(expectation_sum_3step), dtype=np.int32) * 10
            for _ in range(max_event_per_series):  # 高い順に最大max_event_per_series個のeventを決定
                # 3.1 最大値の取得
                # 合計の期待値が最大のstepを取得
                max_step3 = expectation_sum_3step.argmax()
                max_score3 = expectation_sum_3step[max_step3]
                max_step9 = expectation_sum_9step.argmax()
                max_score9 = expectation_sum_9step[max_step9]
                if max_score3 > max_score9:
                    # print('max_score3')
                    left_nums = [0] + high_match_nums
                    right_nums = [0] + low_match_nums
                    max_step_index = max_step3
                    max_score = max_score3
                    result_events_records.append(
                        {
                            "series_id": series_id,
                            "step": max_step_index * 12 + 3,
                            "event": event,
                            "score": max_score,
                        }
                    )
                else:
                    # print('max_score9')
                    left_nums = [0] + low_match_nums
                    right_nums = [0] + high_match_nums
                    max_step_index = max_step9
                    max_score = max_score9
                    result_events_records.append(
                        {
                            "series_id": series_id,
                            "step": max_step_index * 12 + 9,
                            "event": event,
                            "score": max_score,
                        }
                    )
                if max_score < height:  # 閾値以下なら終了
                    break
                # print(f"max_step_index:{max_step_index}, max_score:{max_score}")

                # 3.2 期待値の割引
                """
                各予測地点のpowerを修正するとともに、候補地点の期待値を割引く。
                powerが pi まで小さくなることによってその予測値が影響を与える範囲が狭くなる。
                つまり狭くなって範囲から抜けた expectation_sum の値が、その予測値の値*重みの分だけ小さくなる
                """
                # 3.2.1 まずはpowerを修正するstepの候補を探す
                target_step_powers = []  # (target_step, pred, base_power, power)のリスト
                for pi in range(0, 10):
                    # 左側
                    for l_diff in range(left_nums[pi], left_nums[pi + 1]):
                        target_step_index = max_step_index - l_diff
                        if target_step_index < 0:
                            break
                        pred = preds[target_step_index]
                        base_power = powers[target_step_index]
                        if base_power > pi:  # power が小さくなる場合のみ修正
                            target_step_powers.append((target_step_index, pred, base_power, pi))
                    # 右側
                    for r_diff in range(right_nums[pi] + 1, right_nums[pi + 1] + 1):  # 自分自身と同じindexは含めない
                        target_step_index = max_step_index + r_diff
                        if target_step_index >= len(powers):
                            break
                        pred = preds[target_step_index]
                        base_power = powers[target_step_index]
                        if base_power > pi:
                            target_step_powers.append((target_step_index, pred, base_power, pi))
                # print('target_step_powers', target_step_powers)

                # 3.2.2 対象となる step の power を修正するとともに期待値を割り引く
                """
                powerを下げるとともに、関連する期待値を修正する。
                予測地点に遠いeventを検出した場合は、予測地点に近い候補地点であっても期待値はその分割り引かれる。
                
                3stepの修正をする時は target_stepから左側が low_match_nums, 右側が high_match_nums
                9stepの修正をする時は target_stepから左側が high_match_nums, 右側が low_match_nums
                だんだんと内側のみがのこるように修正する。

                - powerが 10 → 8 になるケースは左右1~30個に影響を及ぼしていたものが、左右の1~20個に影響を及ぼすようになる。また、powerが2個減った分全体の期待値も割り引かれる
                - powerが 10 → 5 になるケースは左右1~30個に影響を及ぼしていたものが、左右の1~12(13)個に影響を及ぼすようになる
                - powerが 8 → 7 になるケースは左右1~20個に影響を及ぼしていたものが、左右の1~15個に影響を及ぼすようになる
                """
                # print(expectation_sum_3step[max_step_index], expectation_sum_9step[max_step_index])
                for si, pred, base_power, power in target_step_powers:
                    # print(f"si:{si}, pred:{pred}, base_power:{base_power}, power:{power}")
                    powers[si] = power
                    # 中心ほど重みが強いので power ごとに処理
                    for pi in range(
                        base_power, power, -1
                    ):  # base_powerからpowerに減らしていくことで予測値の外側から削る
                        # 修正範囲を決定
                        # 3stepの修正をする時は target_step_indexから左側が low_match_nums, 右側が high_match_nums
                        left_nums = [0] + low_match_nums
                        right_nums = [0] + high_match_nums
                        ## left (left_diff_min, left_diff_max] を消す
                        left_diff_max = left_nums[pi]
                        # left_diff_min = left_nums[pi - 1]
                        ## right (right_diff_min, right_diff_max] を消す
                        right_diff_max = right_nums[pi]
                        # right_diff_min = right_nums[(pi - 1)]
                        expectation_sum_3step[si - left_diff_max : si + right_diff_max] -= pred / match_sums[pi-1]
                        """
                        if ((si-left_diff_max <= max_step_index) and (max_step_index < si+right_diff_max)):
                            print(f'3step pi:{pi}')
                            print(f"max_step_index:{max_step_index}, si:{si}, left_diff_max:{left_diff_max}, right_diff_max:{right_diff_max}") 
                            print("[", si-left_diff_max, si+right_diff_max, ")")
                            print(f"power: {pi}→{pi-1}, pred:{pred}")
                            print()
                        """

                        # 9step
                        left_nums = [0] + high_match_nums
                        right_nums = [0] + low_match_nums
                        # [si-left_diff_max, si-left_diff_min)
                        left_diff_max = left_nums[pi]
                        # left_diff_min = left_nums[(pi - 1)]
                        # [si+right_diff_min, si+right_diff_max)
                        # right_diff_min = right_nums[(pi - 1)]
                        right_diff_max = right_nums[pi]
                        expectation_sum_9step[si - left_diff_max : si + right_diff_max] -= pred / match_sums[pi-1]
                        """
                        if ((si-left_diff_max <= max_step_index) and (max_step_index < si+right_diff_max)):
                            print(f'9step pi:{pi}')
                            print(f"max_step_index:{max_step_index}, si:{si}, left_diff_max:{left_diff_max}, right_diff_max:{right_diff_max}") 
                            print("[", si-left_diff_max, si+right_diff_max, ")")
                            print(f"power: {pi}→{pi-1}, pred:{pred}")
                            print()
                        """

                # print(expectation_sum_3step[max_step_index], expectation_sum_9step[max_step_index])

    sub_df = pl.DataFrame(result_events_records).sort(by=["series_id", "step"])
    row_ids = pl.Series(name="row_id", values=np.arange(len(sub_df)))
    sub_df = sub_df.with_columns(row_ids).select(["row_id", "series_id", "step", "event", "score"])
    return sub_df


In [181]:
# 一旦 series1つでのスコア計算テスト
"""
"efbfc4526d58"
"6ca4f4fca6a2"
"a88088855de5"
"3be2f86c3e45"
"e30cb792a2bc"
"18b61dd5aae8"
"e6ddbaaf0639"
"9a340507e36a"
"""
# テストのために一つのseriesに絞る
series_id = "99b829cbad2d" #"99b829cbad2d"
series_df = pred_all_df.filter(pl.col('series_id')==series_id)
series_event_df = event_df.filter(pl.col('series_id')==series_id)

"""
series_df=series_df.with_columns(
    pl.lit(1.0, dtype=pl.Float32).alias('prediction_onset')
)
"""

display(series_event_df.head())

print('new')
sub_df = post_process(
    remove_periodicity(series_df, periodicity_dict),
    event_rate=500, #0.01, # 0.005: 0.8242786502556662
    height = 0.01,
    use_weight=True
) 
display(sub_df.head())
print(len(sub_df))
        
score = event_detection_ap(
    series_event_df.to_pandas(),
    sub_df.to_pandas(),
)

print(score)

print()
print('find_peaks')
sub_df = make_submission(
    series_df,
    periodicity_dict= periodicity_dict,
    height = 0.001,
    distance = 110,
    daily_score_offset = 1.0,
    day_norm=True
) 
display(sub_df.head())
print(len(sub_df))
        
score = event_detection_ap(
    series_event_df.to_pandas(),
    sub_df.to_pandas(),
)

print(score)

series_id,night,event,step,timestamp
str,i64,str,i64,"datetime[μs, UTC]"
"""99b829cbad2d""",1,"""onset""",3960,2017-10-01 01:30:00 UTC
"""99b829cbad2d""",1,"""wakeup""",10836,2017-10-01 11:03:00 UTC
"""99b829cbad2d""",2,"""onset""",20508,2017-10-02 00:29:00 UTC
"""99b829cbad2d""",2,"""wakeup""",27624,2017-10-02 10:22:00 UTC
"""99b829cbad2d""",3,"""onset""",37884,2017-10-03 00:37:00 UTC


new


find peaks:   0%|          | 0/1 [00:00<?, ?it/s]

find peaks:   0%|          | 0/1 [00:00<?, ?it/s]

row_id,series_id,step,event,score
i64,str,i64,str,f64
0,"""99b829cbad2d""",2361,"""onset""",0.011819
1,"""99b829cbad2d""",2751,"""onset""",0.012751
2,"""99b829cbad2d""",2775,"""onset""",0.04116
3,"""99b829cbad2d""",2799,"""onset""",0.102666
4,"""99b829cbad2d""",2829,"""onset""",0.568996


...

find peaks:   0%|          | 0/1 [00:00<?, ?it/s]

row_id,series_id,step,event,score
i64,str,u32,str,f32
0,"""99b829cbad2d""",2363,"""onset""",0.001348
1,"""99b829cbad2d""",2475,"""onset""",0.00078
2,"""99b829cbad2d""",2715,"""onset""",0.00168
3,"""99b829cbad2d""",2825,"""onset""",0.118529
4,"""99b829cbad2d""",2935,"""onset""",0.00163


163


Matching detections to ground truth events:   0%|          | 0/2 [00:00<?, ?it/s]

0.8987719560133354


In [161]:
tmp = remove_periodicity(pred_all_df, periodicity_dict)

with trace("post_process"):
    sub_df = post_process(
        tmp,
        event_per_minute = 0.005,
        height = 0.1,
    ) 
    display(sub_df.head())
print(len(sub_df))

score = event_detection_ap(
    event_df.to_pandas(),
    sub_df.to_pandas(),
)

print(score)

find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

row_id,series_id,step,event,score
i64,str,i64,str,f64
0,"""038441c925bb""",4923,"""onset""",0.148425
1,"""038441c925bb""",4947,"""onset""",0.442321
2,"""038441c925bb""",4971,"""onset""",1.083233
3,"""038441c925bb""",4995,"""onset""",25.857357
4,"""038441c925bb""",5025,"""onset""",0.732165


[25.6GB(+0.7GB):116.3sec] post_process 


228167


Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

0.7922797626687601


In [53]:
sub_df1 = make_submission(
    pred_all_df,
    periodicity_dict= periodicity_dict,
    height = 0.001,
    distance = 107,
    daily_score_offset = 1.0,
    day_norm=True
) 
print(len(sub_df1))

score = event_detection_ap(
    event_df.to_pandas(),
    sub_df1.to_pandas(),
)
print(score) #0.78818291 0.756548
 

find peaks:   0%|          | 0/277 [00:00<?, ?it/s]

46843


Matching detections to ground truth events:   0%|          | 0/538 [00:00<?, ?it/s]

0.8133512719452747
